## Example: Isothermal Flash Calculation

https://pubs.acs.org/doi/pdf/10.1021/ie300183e

Flash calculations are an essential element of models with vapor-liquid equilibrium, including chemical process design, reservoir simulations, environmental modeling.

A feed stream $F$ with composition mole fractions $\{z_i\}_{i=1}^n$ is split into a vapor stream $V$ with composition mole fractions $\{y_i\}_{i=1}^n$, and a liquid stream $L$ with composition mole fractions $\{x_i\}_{i=1}^n$. At equilibrium the vapor and liquid compositions satisfy a relationship

$$
\begin{align*}
y_i & = K_i x_i & \forall i = 1, \dots, n
\end{align*}
$$

A material balance is written for each component

$$
\begin{align*}
F z_i & = L x_i + V y_i & \forall i = 1, \dots, n
\end{align*}
$$

The overall material balance $L = V - F$. Dividing by $F$ and letting $\phi = V/F$ results in $2n$ equations for $2n + 1$ variables $\phi$, $\{x_i\}_{i=1}^n$ and $\{y_i\}_{i=1}^n$

$$
\begin{align*}
z_i & = (1-\phi) x_i + \phi y_i & \forall i = 1, \dots, n \\
y_i & = K_i x_i& \forall i = 1, \dots, n \\
\end{align*}
$$

The problem is fully defined with either $\sum_{i=1}^n x_i = 1$ or $\sum_{i=1}^n x_i = 1$.


### McCormick relaxtions

$$x_i = \frac{z_i}{1 + (K_i - 1)\phi}$$

The isothermal flash model is given by

$$z_i = x_i + (K_i - 1)\phi x_i$$

where $0 \leq x_i \leq 1$ and $0 \leq \phi \leq 1$

Define $w_i = \phi x_i$

$$
\begin{align*}
w_i & \geq 0 \\
w_i & \geq x - y - 1 \\
w_i & \leq x \\
w_i & \leq \phi
\end{align*}
$$

In [ ]:
import pyomo.environ as pyo
import numpy as np

# data
K = np.array([2.0, 1.3, 0.3])
z = np.array([0.3, 0.3, 0.4])

assert len(K) == len(z)
assert any([k > 1 for k in K])
assert any([k < 1 for k in K])
assert sum(z) == 1.0

m = pyo.ConcreteModel()
m.C = pyo.RangeSet(len(K))
m.x = pyo.Var(m.C, domain=pyo.NonNegativeReals)
m.y = pyo.Var(m.C, domain=pyo.NonNegativeReals)
m.f = pyo.Var(bounds=(0, 1))

@m.Param(m.C)
def K(m, c):
    return K[c-1]

@m.Param(m.C)
def z(m, c):
    return z[c-1]

@m.Constraint(m.C)
def VL_equil(m, c):
    return m.y[c] == m.K[c] * m.x[c]

@m.Constraint()
def y_mole_fractions(m):
    return sum(m.y[c] for c in m.C) == 1

@m.Constraint(m.C)
def mass_balance(m, c):
    return m.z[c] == m.x[c] + m.f*(m.y[c] - m.x[c])

pyo.SolverFactory('ipopt').solve(m)

print (m.f())

for c in m.C:
    print(f"{c}) K = {m.K[c]} z = {m.z[c]}  x = {m.x[c]():0.4f}  y = {m.y[c]():0.4f}")


Relationship to a generalized eigenvalue problem.

$$
\begin{align*}
\left(
\begin{bmatrix}
1 & 0 & \dots & 0 & -z_1 \\
0 & 1 & \dots & 0 & -z_2 \\
\vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & \cdots & 1 & -z_n \\
1 & 1 & \cdots & 1 & - 1 \\
\end{bmatrix} -
\phi
\begin{bmatrix}
1 - K_1 & 0 & \cdots & 0 & 0 \\
0 & 1 - K_2 & \cdots & 0 & 0 \\
\vdots & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & \cdots & 1 - K_n & 0 \\
0 & 0 & \cdots & 0 & 0
\end{bmatrix}
\right)
\begin{bmatrix}x_1 \\ x_2 \\ \vdots \\ x_n \\ 1 \end{bmatrix} 
& = 
\begin{bmatrix} 0 \\ 0 \\ \vdots \\ 0 \\ 0 \end{bmatrix}
\end{align*}
$$